In [ ]:
from urllib import request
from lxml import etree
import re
import spacy



nlp = spacy.load('en_core_web_md')
# Define included entities
include_entities = ['ORG']
# Define extract_entities()，利用spacy提取用户所想找的股票名称，打上ORG的标签
def extract_entities(message):
    ents = dict.fromkeys(include_entities)
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            ents[ent.label_] = ent.text
    return ents




def send_message(policy, state, message):
    print("USER : {}".format(message))
    new_state, response = respond(policy, state, message)
    print("BOT : {}".format(response))
    return new_state




def respond(policy, state, message):
    (new_state, response) = policy[(state, interpret(message))]
    return new_state, response



#状态机的状态
def interpret(message):
    msg = message
    if 'stock' in msg:                         #关键词 stock
        return 'stock'
    if item_stock[0]['stock_name'][0] in msg:  #关键词：各个股票的名称缩写（要求大写）
        return 'specify_stock'
    if item_stock[1]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[2]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[3]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[4]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[5]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[6]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[7]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[8]['stock_name'][0] in msg:
        return 'specify_stock'
    if item_stock[9]['stock_name'][0] in msg:
        return 'specify_stock'
    if 'what' in msg:                          #关键词 what
        return 'ask_explanation'
    if 'yes' in msg:                           #关键词 yes
        return 'agree'
    return 'none'



#利用爬虫获取网页信息
#实时获取，股票信息
#web_info = request.urlopen('https://money.cnn.com/data/markets/').read().decode('gbk')
web_info = request.urlopen('https://money.cnn.com/data/us_markets/').read()
#字符串转换为html对象
web_info = etree.HTML(web_info)
#根据网页信息，name of the stock, tbody 不要
#//div[@id='wsod_marketMoversContainer']/table/tbody/tr[2]/td/a/text()
#price
#//div[@id='wsod_marketMoversContainer']/table/tbody/tr[2]/td[2]/span/text()
#change
#//div[@id='wsod_marketMoversContainer']/table/tbody/tr[2]/td[3]/span/span/text()
#percentage change
#//div[@id='wsod_marketMoversContainer']/table/tbody/tr[2]/td[4]/span/span/text()

#创建列表，保存网页上的各支股票数据
item_stock = []
for each in web_info.xpath("//div[@id='wsod_marketMoversContainer']/table//tr"):
#创建字典
    item_stock_each = {}
    if each.xpath("./td/a/text()") != [] :
        item_stock_each['stock_name'] = each.xpath("./td/a/text()")
        item_stock_each['stock_price'] = each.xpath("./td[2]/span/text()")
        item_stock_each['stock_change'] = each.xpath("./td[3]/span/span/text()")
        item_stock_each['stock_percentage'] = each.xpath("./td[4]/span/span/text()")
        item_stock.append(item_stock_each)



        
        
#利用状态机，作为机器人问询股票时候的应答
def stock_function(message):
    #状态机的各个状态
    #初始状态
    INIT = 0
    #选择选择股票
    CHOOSE_STOCK = 1
    #确认哪只股票
    COMFIRM = 2
    #显示数据
    SHOW = 3
    policy = {
        (INIT, "stock"): (CHOOSE_STOCK, "ok, which stock would you like to check?"),  #提到market时，视为询问股票行情
        (INIT, "ask_explanation"): (INIT, "I'm a bot to help you check your stock information."),
        (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
        (CHOOSE_STOCK, "specify_stock"): (COMFIRM, "Do you like to see its price and change?"),
        (CHOOSE_STOCK, "ask_explanation"): (CHOOSE_STOCK,  "i have the information of " +  
                                                            item_stock[0]['stock_name'][0] + " 、 " +
                                                            item_stock[1]['stock_name'][0] + " 、 " +
                                                            item_stock[2]['stock_name'][0] + " 、 " +
                                                            item_stock[3]['stock_name'][0] + " 、 " +
                                                            item_stock[4]['stock_name'][0] + " 、 " +
                                                            item_stock[5]['stock_name'][0] + " 、 " +
                                                            item_stock[6]['stock_name'][0] + " 、 " +
                                                            item_stock[7]['stock_name'][0] + " 、 " +
                                                            item_stock[8]['stock_name'][0] + " 、 " +
                                                            item_stock[9]['stock_name'][0] + " 、 " +
                                                            "  you could also browse website https://money.cnn.com/data/us_markets/ for deatils"),
        (CHOOSE_STOCK, "none"): (CHOOSE_STOCK, "I'm sorry - which stock would you like to check?"),
        (COMFIRM, "agree"): (SHOW, "The information is here !"),
        (COMFIRM, "none"): (CHOOSE_STOCK, "which stock would you like to see"),
    }
    state = INIT       
    while True:
        #在用户输入股票名之后，就是COMFIRM状态，利用spacy提取股票名称
        if state == COMFIRM:
            stock_seeking = extract_entities(message)['ORG']
        if state != SHOW:
        #用户输入信息，在确认要查询哪只股票之前，进行状态机的状态变迁
            message = input("") 
            state =  send_message(policy, state, message)
        else:
            #在确认要查询哪只股票之后，给出信息     
            i = 0
            while True:
                stock_name = item_stock[i]['stock_name'][0]
                if stock_name == stock_seeking:
                    print(item_stock[i]['stock_name'][0] + 
                          ' is at a price of ' + 
                          item_stock[i]['stock_price'][0] + 
                          '  with a change of ' + 
                          item_stock[i]['stock_change'][0] + 
                          ' ,or to say, a percentage change of ' +
                          item_stock[i]['stock_percentage'][0])
                    break
                elif i == len(item_stock):
                    break
                i += 1      
    return response




#主程序
while True:
    message = input("") 
    stock_function(message)